<a href="https://colab.research.google.com/github/ddarshana04/Automatic-Fish-species-detection-and-classification/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Dec 10 02:47:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 00000IMG_00000_BURST20210117180958391_COVER.jpg
'134_3_1313857_1647420754_AWS Course Completion Certificate.pdf'
'13 function overloading.docx'
'14 Text file.docx'
'1581314896392575342449795542983 (1).jpg'
 1581314896392575342449795542983.jpg
'15 Binary file (1).docx'
'15 Binary file.docx'
'16.binary file.docx'
 227a96a3-0277-49e6-9a5d-9a45fcbb4e61.pdf
 2.apk
 3.apk
 3TB_NguyenAnhKiet
 a909713a-ea8e-4643-9702-36da9b3ba6e1.pdf
'Adobe Scan May 18, 2021.pdf'
 Alison_Certificate-1447-16621842.pdf
'Ask Your Doubts!!!!.gform'
 ASSESSMENT1CSE.odt
 Banner-AUTH0-X-HTB-CTF-y09jbjhzaropt3gxoi23gg.jpg
'[bi0s]  Python Advanced Tasks.gdoc'
 Biometrics_Presentation.gslides
 Bioremediation.gslides
 Blockchain.gslides
'bny-cqpq-xus – 7 Jul 2021.gjam'
 B+Tech+Regulations+2019+ammended+May+2020.pdf
 Business.gdoc
 certificate-Be-A-White-Hat-Hacker-and-Pen-Tester-converted.pdf
'Certificate for SANJAI SIDDHARTHAN M for _Entry Level 5 Days Ethical ..._.pdf'
'CH (1).EN.U4CSE19036.

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.14 MiB | 14.75 MiB/s, done.
Resolving deltas: 100% (10406/10406), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "fish" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-12-10 02:49:03--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.4MB/s    in 10s     

2022-12-10 02:49:14 (15.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/00001.png      
  inflating: data/obj/00001.txt      
  inflating: data/obj/1.JPG          
  inflating: data/obj/1.txt          
  inflating: data/obj/00002.png      
  inflating: data/obj/00002.txt      
  inflating: data/obj/2.JPG          
  inflating: data/obj/2.txt          
  inflating: data/obj/00003.png      
  inflating: data/obj/00003.txt      
  inflating: data/obj/3.JPG          
  inflating: data/obj/3.txt          
  inflating: data/obj/00004.png      
  inflating: data/obj/00004.txt      
  inflating: data/obj/4.JPG          
  inflating: data/obj/4.txt          
  inflating: data/obj/00005.png      
  inflating: data/obj/00005.txt      
  inflating: data/obj/5.JPG          
  inflating: data/obj/5.txt          
  inflating: data/obj/00006.png      
  inflating: data/obj/00006.txt      
  inflating: data/obj/6.JPG          
  inflating: data/obj/6.txt          
  inflating: data/obj/00007.png      
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.496094 0.457682 0.199219 0.873698

['0', '0.496094', '0.457682', '0.199219', '0.873698']
0 data/obj/d (27).txt
0 0.496094 0.457682 0.199219 0.873698
0 0.474400 0.405075 0.864760 0.371241

['0', '0.474400', '0.405075', '0.864760', '0.371241']
1 data/obj/3.txt
0 0.474400 0.405075 0.864760 0.371241
0 0.536370 0.395442 0.895480 0.369831

['0', '0.536370', '0.395442', '0.895480', '0.369831']
2 data/obj/4.txt
0 0.536370 0.395442 0.895480 0.369831
0 0.512207 0.589844 0.928711 0.424479

['0', '0.512207', '0.589844', '0.928711', '0.424479']
3 data/obj/w (19).txt
0 0.512207 0.589844 0.928711 0.424479
0 0.438383 0.571898 0.155014 0.613722

['0', '0.438383', '0.571898', '0.155014', '0.613722']
4 data/obj/s (29).txt
0 0.438383 0.571898 0.155014 0.613722
0 0.547852 0.465495 0.816406 0.259115

['0', '0.547852', '0.465495', '0.816406', '0.259115']
5 data/obj/00023.txt
0 0.547852 0.465495 0.816406 0.259115
0 0.525879 0.494792 0.948242 0.476562

['0', '0.525879', '0.494792', '0.948242', '0.476562']


In [ ]:
import glob
images_list = glob.glob("data/obj/*.JPG")
images_list += glob.glob("data/obj/*.png")
print(images_list)

['data/obj/4.JPG', 'data/obj/7.JPG', 'data/obj/100_1480.JPG', 'data/obj/29.JPG', 'data/obj/17.JPG', 'data/obj/SAM_5549.JPG', 'data/obj/SAM_5537.JPG', 'data/obj/100_1462.JPG', 'data/obj/23.JPG', 'data/obj/100_1484.JPG', 'data/obj/30.JPG', 'data/obj/100_1489.JPG', 'data/obj/SAM_5538.JPG', 'data/obj/SAM_5533.JPG', 'data/obj/100_1461.JPG', 'data/obj/14.JPG', 'data/obj/SAM_5552.JPG', 'data/obj/100_1487.JPG', 'data/obj/18.JPG', 'data/obj/5.JPG', 'data/obj/1.JPG', 'data/obj/16.JPG', 'data/obj/SAM_5547.JPG', 'data/obj/100_1483.JPG', 'data/obj/100_1465.JPG', 'data/obj/SAM_5540.JPG', 'data/obj/100_1486.JPG', 'data/obj/100_1463.JPG', 'data/obj/100_1488.JPG', 'data/obj/19.JPG', 'data/obj/3.JPG', 'data/obj/100_1476.JPG', 'data/obj/25.JPG', 'data/obj/100_1485.JPG', 'data/obj/12.JPG', 'data/obj/100_1479.JPG', 'data/obj/SAM_5534.JPG', 'data/obj/2.JPG', 'data/obj/26.JPG', 'data/obj/SAM_5550.JPG', 'data/obj/SAM_5536.JPG', 'data/obj/27.JPG', 'data/obj/SAM_5535.JPG', 'data/obj/8.JPG', 'data/obj/100_1481.J

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.202093), count: 4, class_loss = 1.472002, iou_loss = 3.938097, total_loss = 5.410099 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.362529, iou_loss = 0.000000, total_loss = 0.362529 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.344538, iou_loss = 0.000000, total_loss = 0.344538 
 total_bbox = 15078, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.352388), count: 4, class_loss = 1.532127, iou_loss = 2.555517, total_loss = 4.087644 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.358103, iou_loss = 0.000000, total_loss = 0.358103 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00

In [ ]:
from google.colab import drive
drive.mount('/content/drive')